In [1]:
import os
import glob
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
# curl https://natura.di.uminho.pt/~jj/pln/calao/dicionario.html > dicionario.html

with open("dicionario.html") as fp:
    soup = BeautifulSoup(fp)


In [15]:
terms = div.find("div", attrs = {"class": "inf"})
slang_df = pd.DataFrame(terms)
slang_df

,0
0,"[\n, [bater, normalmente usado como ameaça de ..."
1,\n


In [46]:
divs = soup.findAll(class_= 'entry')    

dict_table = pd.DataFrame()

for div in divs:
    #term
    entry_details = {}
    term = div.find("div", attrs = {"class": "term"}).string
    entry_details["term"] = [term]
    
    #description
    description_key = div.find("div", attrs = {"class": "inf"})
    description = description_key and description_key.find("dt")
    entry_details["description"] = [description and description.string]

    def add_if_exists(element, column):
        entry_details[column] = [
            element.next_sibling.string if element is not None
            else "none"] 
    
    #levels
    add_if_exists(div.find("dt", string="level:"), "level")

    #synonyms
    sinonyms_key = div.find("i", string="sin.:")
    if sinonyms_key:
        sinonyms_key = sinonyms_key.parent
    add_if_exists(sinonyms_key, "synonym")
    
    entry_details = pd.DataFrame.from_dict(entry_details)
    dict_table = pd.concat([dict_table, entry_details])

dict_table

,term,description,level,synonym
0,a altas horas da noite,muito tarde na noite,none,noite alta; noite cerrada; lá para as quinhentas
0,abafa-palhinha,&bullet; homossexual masculino passivo,calão carroceiro,arrombado; paneleiro; larilas; lelé; bicha; b...
0,abafar,"esconder, esquecer, mudar de assunto, fazer co...",none,none
0,abafo,"festas, carinho",none,cafuné; carícia; meiguice; quitute; quindim; ...
0,abananado,"admirado, aturdido e sem palavras",coloquial,estupefacto; azuratado; aturdido; embasbacado...
...,...,...,...,...
0,zombi,"pessoa que, mesmo a pé, está sempre a dormir",none,dorminhoco; sonolento; cu de sono
0,zuate,"referente à região das nádegas ou ânus, freque...",calão,bunda; alcofa; cu; peida; rabo; bilha; trasei...
0,zuca,é um:,none,brazuca
0,zumba que zumba,None,none,none
